In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset train




In [ ]:
trainNews = pd.read_csv('/content/New_News_Test.csv')

In [ ]:
X_trainnews = trainNews['Removed_stopwords']
y_trainnews = trainNews['Type']

In [ ]:
trainNews

,Type,Removed_stopwords
0,1,مئات القتلى فيضانات اوروبا
1,1,جولة مفاوضات جديدة تفشل كسر جمود الازمة القبرصية
2,1,القضاء الفرنسي يفتح تحقيقا حول بيغاسوس بشان ال...
3,1,وزير الدفاع الاسرائيلي يزور فرنسا لبحث قضية ″ب...
4,1,بدا عدة دول سريان الحظر الشامل للحد انتشار فير...
...,...,...
995,3,هواوي العقوبات الاميركية الشركة مسؤولة جزئيا ا...
996,3,مجهر الهواتف المحمولة القديمة للمدارس تايلاند
997,3,هواتف ذكية شبيهة بالليجو تشق طريقها الى الاسواق
998,3,هواوي واشنطن تضيف شركة شياومي الصينية الى القا...


In [ ]:
trainNews2 = pd.read_csv('/content/New_News_Test.csv', delimiter='\t', header=None)

In [ ]:
trainNews2

,0
0,"Type,Removed_stopwords"
1,"1,مئات القتلى فيضانات اوروبا"
2,"1,جولة مفاوضات جديدة تفشل كسر جمود الازمة القب..."
3,"1,القضاء الفرنسي يفتح تحقيقا حول بيغاسوس بشان ..."
4,"1,وزير الدفاع الاسرائيلي يزور فرنسا لبحث قضية ..."
...,...
996,"3,هواوي العقوبات الاميركية الشركة مسؤولة جزئيا..."
997,"3,مجهر الهواتف المحمولة القديمة للمدارس تايلاند"
998,"3,هواتف ذكية شبيهة بالليجو تشق طريقها الى الاسواق"
999,"3,هواوي واشنطن تضيف شركة شياومي الصينية الى ال..."


In [ ]:
trainNews2 = trainNews2.iloc[1:]

In [ ]:
trainNews2

,0
1,"1,مئات القتلى فيضانات اوروبا"
2,"1,جولة مفاوضات جديدة تفشل كسر جمود الازمة القب..."
3,"1,القضاء الفرنسي يفتح تحقيقا حول بيغاسوس بشان ..."
4,"1,وزير الدفاع الاسرائيلي يزور فرنسا لبحث قضية ..."
5,"1,بدا عدة دول سريان الحظر الشامل للحد انتشار ف..."
...,...
996,"3,هواوي العقوبات الاميركية الشركة مسؤولة جزئيا..."
997,"3,مجهر الهواتف المحمولة القديمة للمدارس تايلاند"
998,"3,هواتف ذكية شبيهة بالليجو تشق طريقها الى الاسواق"
999,"3,هواوي واشنطن تضيف شركة شياومي الصينية الى ال..."


### **Importing the dataset test**

In [ ]:
testNews = pd.read_csv("/content/New_News_Test.csv")

In [ ]:
X_testnews = testNews['Removed_stopwords']
y_testnews = testNews['Type']

In [ ]:
testNews2 = pd.read_csv('/content/New_News_Test.csv', delimiter='\t', header=None)

In [ ]:
testNews2 = testNews2.iloc[1:]

For performance reasons, we'll only use 2,000 sentences from the dataset

In [ ]:
batch_1 = trainNews2[:2000]

In [ ]:
batch_2 = testNews2[:1000]

## **Loading the Pre-trained BERT model**

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'aubmindlab/bert-base-arabertv02')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight'

### **Tokenization**

In [ ]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
tokenized2 = batch_2[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

### **Padding**

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
max_len = 0
for i in tokenized2.values:
    if len(i) > max_len:
        max_len = len(i)

padded2 = np.array([i + [0]*(max_len-len(i)) for i in tokenized2.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

(1000, 28)

In [ ]:
np.array(padded2).shape

(999, 28)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 28)

In [ ]:
attention_mask2 = np.where(padded2 != 0, 1, 0)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
input_ids2 = torch.tensor(padded2)
attention_mask2 = torch.tensor(attention_mask2)

with torch.no_grad():
    last_hidden_states2 = model(input_ids2, attention_mask=attention_mask2)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features2 = last_hidden_states2[0][:,0,:].numpy()

In [ ]:
features

array([[ 0.18965468,  1.1767994 , -0.81534135, ..., -0.5349166 ,
        -1.1029911 , -2.0686066 ],
       [-0.23016177,  0.7650301 , -0.15764838, ..., -0.163252  ,
        -1.0426102 , -1.9175332 ],
       [-0.4406399 ,  1.2645612 , -0.34343317, ..., -0.6080211 ,
        -0.82027364, -1.6830822 ],
       ...,
       [-0.6603649 ,  1.0452776 ,  0.04258461, ..., -0.253115  ,
        -0.88264465, -0.98000616],
       [-0.619232  ,  0.89525807, -0.07310441, ...,  0.14004971,
        -0.7665214 , -1.5596198 ],
       [-0.7184792 ,  1.4211073 , -0.13141146, ..., -0.08940903,
        -1.0010604 , -2.14059   ]], dtype=float32)

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [ ]:
labels = y_trainnews

In [ ]:
labels2 = y_testnews[1:]

# **Training on naive bayes:**


In [ ]:
# prompt: train a nive bayes model

from sklearn.naive_bayes import GaussianNB
NBmodel = GaussianNB()
NBmodel.fit(features, labels)


GaussianNB()

In [ ]:
import numpy as np

predicted_labels = NBmodel.predict(features)

# Calculate the accuracy of the model
accuracy = np.mean(predicted_labels == labels)
print("Accuracy:", accuracy)


Accuracy: 0.987


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(labels, predicted_labels))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.97       200
           1       0.99      1.00      0.99       512
           2       0.99      0.99      0.99       200
           3       0.97      1.00      0.98        88

    accuracy                           0.99      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.99      0.99      0.99      1000



In [ ]:
predicted_labels2 = NBmodel.predict(features2)

In [ ]:
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(labels2, predicted_labels2))

Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.98      0.34       200
           1       0.58      0.03      0.06       511
           2       0.00      0.00      0.00       200
           3       0.00      0.00      0.00        88

    accuracy                           0.21       999
   macro avg       0.19      0.25      0.10       999
weighted avg       0.34      0.21      0.10       999



### **Training on LSTM:**

In [ ]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Reshape


LSTMmodel = Sequential()

LSTMmodel.add(Reshape((1, features.shape[1]), input_shape=(features.shape[1],)))
LSTMmodel.add(Bidirectional(LSTM(units=64)))


LSTMmodel.add(Dense(1, activation='softmax'))

LSTMmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
LSTMmodel.fit(features, labels, epochs=10, batch_size=32, validation_split=0.2)

LSTMmodel.summary()

Epoch 1/10
25/25 [==============================] - 13s 60ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
25/25 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6400 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+

### **Refrences**

https://github.com/VincentK1991/BERT_summarization_1/blob/master/notebook/Primer_to_BERT_extractive_summarization_March_25_2020.ipynb

https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/